From https://docs.pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html


In [1]:
import pathlib

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data
from torchvision import datasets, transforms

In [2]:
# data loader
BATCH_SIZE = 64
# nn
W1_NUM_NEURONS = 512
W2_NUM_NEURONS = 512
NUM_CLASSES = 10
# optim
LEARNING_RATE = 1e-3
# training
NUM_EPOCHS = 5
# inference
INFERENCE_SAMPLE = 320

In [3]:
# download data
training_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=transforms.ToTensor())

100%|██████████| 26.4M/26.4M [00:06<00:00, 3.79MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 246kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 4.33MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 74.7MB/s]


In [4]:
# create data loaders
train_dataloader = data.DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = data.DataLoader(test_data, batch_size=BATCH_SIZE)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
X_c, X_h, X_w = X.shape[1:]

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else "cpu"
print(f"Using device: {device}")


class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(X_h * X_w, W1_NUM_NEURONS),
            nn.ReLU(),
            nn.Linear(W1_NUM_NEURONS, W2_NUM_NEURONS),
            nn.ReLU(),
            nn.Linear(W2_NUM_NEURONS, NUM_CLASSES),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

Using device: cuda
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [7]:
def train(
    dataloader: data.DataLoader,
    model: nn.Module,
    loss_fn: nn.CrossEntropyLoss,
    optimizer: torch.optim.Optimizer,
) -> None:
    size = len(dataloader.dataset)
    model.train()  # set train mode
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # forward
        logits = model(X)
        loss = loss_fn(logits, y)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(
    dataloader: data.DataLoader,
    model: nn.Module,
    loss_fn: nn.CrossEntropyLoss,
) -> None:
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            logits = model(X)
            test_loss += loss_fn(logits, y).item()
            correct += (logits.argmax(dim=1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 8.5%, Avg loss: 2.307575 



In [8]:
for t in range(NUM_EPOCHS):
    print(f"Epoch {t + 1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------
loss: 2.311560  [   64/60000]
loss: 2.296028  [ 6464/60000]
loss: 2.277280  [12864/60000]
loss: 2.271030  [19264/60000]
loss: 2.259848  [25664/60000]
loss: 2.228582  [32064/60000]
loss: 2.242509  [38464/60000]
loss: 2.210972  [44864/60000]
loss: 2.211518  [51264/60000]
loss: 2.178780  [57664/60000]
Test Error: 
 Accuracy: 32.0%, Avg loss: 2.174878 

Epoch 2
-------------------------------
loss: 2.188722  [   64/60000]
loss: 2.180620  [ 6464/60000]
loss: 2.126408  [12864/60000]
loss: 2.141188  [19264/60000]
loss: 2.103229  [25664/60000]
loss: 2.043301  [32064/60000]
loss: 2.075277  [38464/60000]
loss: 2.002295  [44864/60000]
loss: 2.014890  [51264/60000]
loss: 1.945434  [57664/60000]
Test Error: 
 Accuracy: 52.1%, Avg loss: 1.941720 

Epoch 3
-------------------------------
loss: 1.974746  [   64/60000]
loss: 1.950949  [ 6464/60000]
loss: 1.839235  [12864/60000]
loss: 1.877493  [19264/60000]
loss: 1.774934  [25664/60000]
loss: 1.720471  [32064/600

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[INFERENCE_SAMPLE][0], test_data[INFERENCE_SAMPLE][1]
with torch.no_grad():
    x = x.to(device)
    logits = model(x)
    print(logits)
    predicted, actual = classes[logits[0].argmax(dim=0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")

tensor([[-0.1922, -0.9881,  0.1150, -0.5861, -0.1211,  0.4942,  0.0322,  0.1644,
          0.7092,  0.4481]], device='cuda:0')
Predicted: Bag, Actual: Bag


In [12]:
pathlib.Path("model.pth").unlink(missing_ok=True)